<a href="https://colab.research.google.com/github/swapnilbetkar/python/blob/Dev_Self/SVMHardMargin_ChurnPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.svm import SVC
warnings.filterwarnings('ignore')

In [19]:
#import dataset
dataset = pd.read_csv("/content/drive/MyDrive/Datasets/ChurnPrediction.csv")
pd.set_option('display.max_columns',None)
dataset.head()
# print(dataset.info())

,Age,PastEmployee,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,46,No,Travel_Rarely,Human Resources,5,2,Medical,2,Male,3,2,Human Resources,2,Divorced,5021,8,Yes,22,4,1,16,2,3,4,2,0,2
1,37,Yes,Travel_Rarely,Human Resources,6,4,Human Resources,3,Male,3,1,Human Resources,1,Divorced,2073,4,Yes,22,4,0,7,3,3,3,2,0,2
2,59,No,Non-Travel,Human Resources,2,4,Human Resources,3,Female,2,5,Manager,4,Married,18844,9,No,21,4,1,30,3,3,3,2,2,2
3,54,No,Non-Travel,Human Resources,26,3,Human Resources,4,Female,4,4,Manager,4,Single,17328,2,Yes,12,3,0,23,3,3,5,3,4,4
4,26,No,Travel_Rarely,Human Resources,25,1,Life Sciences,3,Female,3,1,Human Resources,3,Married,2942,1,No,23,4,1,8,3,3,8,7,5,7


In [20]:
# Converting Categorical features into Numerical features
def converter(column):
    if column == 'Yes':
        return 1
    else:
        return 0

dataset['PastEmployee'] = dataset['PastEmployee'].apply(converter)
dataset['OverTime'] = dataset['OverTime'].apply(converter)
# We can also write - dataset['OverTime'] = dataset['OverTime'].map({'Yes':1, 'No':0})
dataset['Gender'] = dataset['Gender'].map({'Female':1, 'Male':0})
print(dataset.info())

categorical_features = ['BusinessTravel', 'Department','EducationField', 'JobRole', 'MaritalStatus']
final_data = pd.get_dummies(dataset, columns = categorical_features)
print(final_data.info())
print(final_data.head(2))

# Dividing dataset into label and feature sets
X = final_data.drop('PastEmployee', axis = 1) # Features
Y = final_data['PastEmployee'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1470 non-null   int64 
 1   PastEmployee             1470 non-null   int64 
 2   BusinessTravel           1470 non-null   object
 3   Department               1470 non-null   object
 4   DistanceFromHome         1470 non-null   int64 
 5   Education                1470 non-null   int64 
 6   EducationField           1470 non-null   object
 7   EnvironmentSatisfaction  1470 non-null   int64 
 8   Gender                   1470 non-null   int64 
 9   JobInvolvement           1470 non-null   int64 
 10  JobLevel                 1470 non-null   int64 
 11  JobRole                  1470 non-null   object
 12  JobSatisfaction          1470 non-null   int64 
 13  MaritalStatus            1470 non-null   object
 14  MonthlyIncome            1470 non-null  

In [21]:
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', SVC() )
    ])

# grid_param = {'classification__n_estimators': [30,40,50,100,150]}
grid_param = {'classification__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'classification__C' : [0.1,0.01,0.001,0.0001]}
gd_sr = GridSearchCV(estimator=model,  param_grid=grid_param,scoring='recall', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print(best_result)

{'classification__C': 0.001, 'classification__kernel': 'sigmoid'}
0.8312056737588651
